In [ ]:
import os
from dotenv import load_dotenv
import pdfplumber
import re
import json
import tiktoken
import spacy

# ==============================================
# 1. Configuração do ambiente
# ==============================================
# Carrega variáveis de ambiente do arquivo .env (ex: PDF_FOLDER e OUTPUT_FOLDER)
load_dotenv()
pdf_folder = os.getenv("PDF_FOLDER")
output_folder = os.getenv("OUTPUT_FOLDER")

# ==============================================
# 2. Funções de limpeza e extração de entidades
# ==============================================
def clean_text(text):
    """
    Limpa o texto extraído do PDF.
    - Substitui quebras de linha por espaço.
    - Remove espaços extras consecutivos.
    - Remove '__' que podem aparecer como lixo no PDF.
    - Corrige quebras no meio de números, como '28. 913' -> '28.913'.
    - Remove marcações de página, ex: "Página 1 de 12".
    """
    if not text:
        return ""
    
    text = re.sub(r'\n+', ' ', text)  # remove múltiplas quebras de linha
    text = re.sub(r'\s+', ' ', text)  # normaliza espaços
    text = re.sub(r'__', '', text)    # remove '__'
    text = re.sub(r'(\d{2,3})\.\s+(\d{3})', r'\1.\2', text)  # corrige números quebrados
    text = re.sub(r'Página\s+\d+\s+de\s+\d+', '', text, flags=re.IGNORECASE)  # remove "Página X de Y"
    return text.strip()

def extract_entities_regex(text):
    """
    Extrai documentos do texto usando expressões regulares:
    - CNPJs: 00.000.000/0000-00
    - CPFs: 000.000.000-00
    - RGs: 00.000.000-0
    - CEPs: 00000-000
    """
    cnpjs = re.findall(r'\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2}', text)
    cpfs  = re.findall(r'\d{3}\.\d{3}\.\d{3}-\d{2}', text)
    rgs   = re.findall(r'\d{1,2}\.\d{3}\.\d{3}-\d{1}', text)
    ceps  = re.findall(r'\d{5}-\d{3}', text)
    return cnpjs + cpfs + rgs + ceps

# ==============================================
# 3. Configuração para chunking por tokens
# ==============================================
all_chunks = []          # lista para armazenar todos os chunks
global_chunk_number = 0  # contador global de chunks
max_tokens = 500         # tamanho máximo do chunk em tokens
overlap = 30             # sobreposição de tokens entre chunks

encoding = tiktoken.get_encoding("cl100k_base")  # encoding usado pelo modelo OpenAI

def text_to_tokens(text):
    """Converte texto em tokens"""
    return encoding.encode(text)

def tokens_to_text(tokens):
    """Converte tokens de volta em texto"""
    return encoding.decode(tokens)

def chunk_text_by_tokens(text, max_tokens, overlap):
    """
    Divide o texto em chunks de tamanho máximo 'max_tokens' com sobreposição 'overlap'.
    - Mantém chunks menores que o overlap se contiverem entidades (CPF, CNPJ, etc.).
    """
    tokens = text_to_tokens(text)
    chunks = []
    start = 0
    while start < len(tokens):
        end = min(start + max_tokens, len(tokens))
        chunk_tokens = tokens[start:end]
        chunk_text = tokens_to_text(chunk_tokens)
        # Ignora chunks menores que o overlap, exceto se tiver entidades
        if len(chunk_tokens) >= overlap or extract_entities_regex(chunk_text):
            chunks.append(chunk_text)
        start += max_tokens - overlap  # avança com sobreposição
    return chunks

# ==============================================
# 4. NER com spaCy (Português)
# ==============================================
nlp = spacy.load("pt_core_news_sm")

def extract_spacy_entities(text):
    """
    Extrai somente entidades de Pessoas (PER) e Organizações (ORG) com spaCy.
    Remove duplicados.
    """
    doc = nlp(text)
    entities = [ent.text for ent in doc.ents if ent.label_ in ("PER", "ORG")]
    return list(set(entities))  # remove duplicados

# ==============================================
# 5. Processamento dos PDFs
# ==============================================
for pdf_file in os.listdir(pdf_folder):
    if not pdf_file.lower().endswith(".pdf"):
        continue  # ignora arquivos que não são PDF
    pdf_path = os.path.join(pdf_folder, pdf_file)
    
    full_text = ""  # acumula o texto inteiro do documento
    
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                page_text = clean_text(page_text)
                full_text += " " + page_text  # acumula texto do documento
                chunks = chunk_text_by_tokens(page_text, max_tokens=max_tokens, overlap=overlap)
                
                for chunk in chunks:
                    # Adiciona chunk original com metadados
                    all_chunks.append({
                        "chunk_number": global_chunk_number,
                        "text": chunk,
                        "source_file": pdf_file,
                        "page_number": i
                    })
                    global_chunk_number += 1
                    
                    # Cria chunk apenas com entidades de regex
                    entities = extract_entities_regex(chunk)
                    if entities:
                        entity_text = " ".join(entities)
                        all_chunks.append({
                            "chunk_number": global_chunk_number,
                            "text": entity_text,
                            "source_file": pdf_file,
                            "page_number": i
                        })
                        global_chunk_number += 1
    
    # 🔹 Depois de processar todas as páginas, aplica NER ao documento inteiro
    spacy_entities = extract_spacy_entities(full_text)
    if spacy_entities:
        all_chunks.append({
            "chunk_number": global_chunk_number,
            "text": " ".join(spacy_entities),
            "source_file": pdf_file,
            "page_number": None   # NER é do documento inteiro
        })
        global_chunk_number += 1

# ==============================================
# 6. Resultados
# ==============================================
print(f"Total de chunks gerados: {len(all_chunks)}")
print("Exemplo de chunk com metadados:\n", all_chunks[0])

# ==============================================
# 7. Salvar em json
# ==============================================
os.makedirs(output_folder, exist_ok=True)
output_file = os.path.join(output_folder, "chunks.json")

with open(output_file, "w", encoding="utf-8") as f:
    json.dump(all_chunks, f, ensure_ascii=False, indent=4)

print(f"✅ {len(all_chunks)} chunks salvos em {output_file}")


Total de chunks gerados: 60
Exemplo de chunk com metadados:
 {'chunk_number': 0, 'text': 'CONTRATO DE ASSOCIAÇÃO DE EMPRESA Pelo presente instrumento e na melhor forma de direito, de um lado, AJJ SERVIÇOS CONTABÉIS LTDA, pessoa jurídica de direito privado, inscrita no CNPJ sob o nº 28.913.706/0001-63, com sede na Rua Alfred Charvet, nº 710, Bairro Vila Nova, Araucária/PR, CEP: 83.703-278, neste ato, representada por seu sócio administrador, Jhonny Cézar de Jesus Falavinha, brasileiro, portador do RG nº 6.223.402-4 SSP/PR, inscrito no CPF nº 007.724.809-07, doravante denominada EMPRESA ASSOCIADA, e de outro lado, o PARQUE CIENTÍFICO E TECNOLÓGICO DE BIOCIÊNCIAS LTDA, pessoa jurídica de direito privado, inscrito no CNPJ sob o nº 21.526.709/0001-03, com sede na Rodovia PR-182, Km 320/321, s/n, Condomínio Industrial Biopark, no município de Toledo, Estado do Paraná, CEP 85.919-899, neste ato representada por seu sócio administrador, Victor Donaduzzi, brasileiro, inscrito no CPF sob o nº 04